In [3]:
import os, time, pickle, argparse, networks, utils
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import transforms
from edge_promoting import edge_promoting

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = True

In [5]:
project_name = 'MyC1'

In [6]:
if not os.path.isdir(os.path.join(project_name + '_results', 'Reconstruction')):
    os.makedirs(os.path.join(project_name + '_results', 'Reconstruction'))
if not os.path.isdir(os.path.join(project_name + '_results', 'Transfer')):
    os.makedirs(os.path.join(project_name + '_results', 'Transfer'))

In [7]:
class discriminator(nn.Module):
    # initializers
    def __init__(self, input_channel, output_channel, n_layers = 5, hidden_size = 32):
        super(discriminator, self).__init__()
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.hidden_size = hidden_size
        self.convs = []
        
        self.convs.append(nn.Conv2d(input_channel, hidden_size, kernel_size = 3, stride = 1, padding = 1))
        current_channel = hidden_size
        
        for i in range(n_layers):
            self.convs.append(nn.Conv2d(current_channel, current_channel*2, kernel_size = 3, stride = 2, padding = 1))
            self.convs.append(nn.LeakyReLU(0.2, True))
            
            self.convs.append(nn.Conv2d(current_channel*2, current_channel*4, kernel_size = 3, stride = 1, padding = 1))
            nn.InstanceNorm2d(current_channel * 4),
            self.convs.append(nn.LeakyReLU(0.2, True))
            # enlarge the channel by 2 times every 2 layers
            current_channel *= 4
        
        self.conv_layers = nn.ModuleList(self.convs)

        utils.initialize_weights(self)

    # forward method
    def forward(self, input):
        # input = torch.cat((input1, input2), 1)
        output = self.conv_layers(input)

        return output


In [8]:
class ConvInstanceNormLeakyRelu(nn.Module):
    def __init__(self, input_channel, output_channel, kernel_size, stride, padding, leaky_rate = 0.2):
        super(ConvInstanceNormLeakyRelu, self).__init__()
        self.conv = nn.Conv2d(input_channel, out_channels, kernel_size = kernel_size, stride = stride, padding = padding)
        self.InstanceNorm = nn.InstanceNorm2d(output_channel)
        self.relu = nn.LeakyReLU(leaky_rate)
    
    def forward(self, x):
        return self.relu(self.InstanceNorm(self.conv(x)))

In [9]:
class resnet_block(nn.Module):
    def __init__(self, channel, kernel, stride, padding):
        super(resnet_block, self).__init__()
        self.channel = channel
        self.kernel = kernel
        self.strdie = stride
        self.padding = padding
        self.conv1 = nn.Conv2d(channel, channel, kernel, stride, padding)
        self.conv1_norm = nn.InstanceNorm2d(channel)
        self.conv2 = nn.Conv2d(channel, channel, kernel, stride, padding)
        self.conv2_norm = nn.InstanceNorm2d(channel)

        utils.initialize_weights(self)

    def forward(self, x):
        identity = x.copy()
        x = F.relu(self.conv1_norm(self.conv1(x)), True)
        x = self.conv2_norm(self.conv2(x))

        return x + identity #Elementwise Sum

In [10]:
def conv_dw(inp, oup, stride):
    return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

In [11]:
class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, nout):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=3, padding=1, groups=nin)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [12]:
class depthwise_separable_conv_stride2(nn.Module):
    def __init__(self, nin, nout):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=3, padding=1, stride = 2, groups=nin)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [16]:
class generator(nn.Module):
    # initializers
    def __init__(self, input_channel, output_channel, n_block_layers = 8, hidden_size = 64):
        super(generator, self).__init__()
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.hidden_size = hidden_size
        
        # The generator have three parts: 1. down samples 2. resnet blocks  3. up samples
        self.n_block_layers = n_block_layers
        
        self.down_convs = [ConvInstanceNormLeakyRelu(input_channel, hidden_size, 3, 1, 3), \
                           ConvInstanceNormLeakyRelu(hidden_size, hidden_size, 3, 1, 1), \
                           depthwise_separable_conv(hidden_size, hidden_size*2), \
                           ConvInstanceNormLeakyRelu(hidden_size*2, hidden_size*2, 3, 1, 1), \
                           depthwise_separable_conv(hidden_size*2, hidden_size*2), \
                           depthwise_separable_conv(hidden_size*2, hidden_size*4), \
                           ConvInstanceNormLeakyRelu(hidden_size*4, hidden_size*4, 3, 1, 1)]
        

        self.resnet_blocks = []
        for i in range(n_block_layers):
            self.resnet_blocks.append(resnet_block(hidden_size * 4, 3, 1, 1))


        # basically, nn.ConvTranspose2d is more adopted in GANs than nn.Upsample
        self.up_convs = [nn.ConvTranspose2d(hidden_size*4, hidden_size*4, 3, 2, 1, 1), #k3n128s1/2
                            nn.Conv2d(nf * 2, nf * 2, 3, 1, 1), #k3n128s1
                            nn.InstanceNorm2d(nf * 2),
                            nn.LeakyReLU(leaky_rate),
                            nn.ConvTranspose2d(nf * 2, nf, 3, 2, 1, 1), #k3n64s1/2
                            nn.Conv2d(nf, nf, 3, 1, 1), #k3n64s1
                            nn.InstanceNorm2d(nf),
                            nn.LeakyReLU(leaky_rate),
                            nn.Conv2d(nf, out_nc, 7, 1, 3), #k7n3s1
                            nn.Hardtanh()]
        
        
        self.down_convs_layers = nn.Sequential(*self.down_convs)
        self.resnet_blocks_layers = nn.Sequential(*self.resnet_blocks)
        self.up_convs_layers = nn.Sequential(*self.up_convs)

        utils.initialize_weights(self)

    # forward method
    def forward(self, input):
        x = self.down_convs_layers(input)
        x = self.resnet_blocks_layers(x)
        output = self.up_convs_layers(x)

        return output

In [ ]:
if not os.path.isdir(os.path.join('data', tgt_data_dir, 'pair')):
    print('edge-promoting start!!')
    edge_promoting(os.path.join('data', tgt_data_dir, 'train'), os.path.join('data', args.tgt_data, 'pair'))
else:
    print('edge-promoting already done')

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import cv2
# Create a black image
image = np.zeros((512,512,3), np.uint8)
# Draw a diagonal blue line with thickness of 5 px
for i in range(100):
    i = random.randint(0, 512)
    j = random.randint(0, 512)
    k = random.randint(0, 14)-7
    l = random.randint(0, 10)-5
    
    cv2.line(image,(i,j),(i+k,j+l),(255, 0, 0),2)

img = Image.fromarray(image, 'RGB')
img.save('my.png')
img.show()